In [3]:
## Import the libraries

import numpy as np
import numpy.ma as ma
from scipy.io import netcdf
from datetime import datetime, timedelta
import matplotlib.dates as mdates
import matplotlib.pyplot as plt  # plotting
from sympy import * 
from __future__ import division
import bodc_data_tools
import time

init_printing()  # for sympy
%matplotlib inline

## Personal librairies
import sys
# the mock-0.3.1 dir contains testcase.py, testutils.py & mock.py
sys.path.append('/login/jelt/matlab/mfiles/ipynb/fastnet')
import bodc_data_tools
import h5py
import amm60_data_tools


######### Personal Paths #############
HOME = '/media/karen/data1/'
CRUISE_DIR = HOME + 'AMM60/FASTNEt/BODC_fastnet/'
AMM60_DIR = HOME + 'AMM60/RUN_nodiff_2012/'
AMM7_DIR = HOME + 'AMM7/2012/'
#NNA_DIR = HOME + 'NNA/2012/'
#CSV_FILENAME = 'bodc_series_metadata_summary.csv'
WRITEDIR = HOME + 'AMM60/FASTNEt'
MOORING_NAMES = ['ST1', 'ST2' ,'ST4' ,'ST5','SE']


# Path fixes while developing and debugging #
import getpass
user = getpass.getuser()
print(user)

if 'jelt' in user:
    HOME = '/projectsa/FASTNEt/kariho40/OBS/BODC_FASTNEt_data/'
    MOORINGmod_filename = '/projectsa/FASTNEt/kariho40/AMM60/RUNS/D376/AMM60_1h_fastnet_ST1.nc'
    MOORINGobs_filename = '/scratch/jelt/mooring_0.nc'
    MOORING_merged_filename = '/scratch/jelt/mooring_ST1_temp.nc'
elif 'karen' in user:
    ######### Personal Paths #############
    HOME = '/media/karen/data1/'
    CRUISE_DIR = HOME + 'AMM60/FASTNEt/BODC_fastnet/'
    AMM60_DIR = HOME + 'AMM60/FASTNEt/'
    AMM7_DIR = HOME + 'AMM7/2012/'
    #NNA_DIR = HOME + 'AMM60/FASTNEt/NNA/'
    NNA_DIR = HOME + 'AMM60/FASTNEt/BODC_fastnet/'
    #CSV_FILENAME = 'bodc_series_metadata_summary.csv'
    WRITEDIR = HOME + 'AMM60/FASTNEt/'
    MOORING_NAMES = ['ST1', 'ST2' ,'ST4' ,'ST5','SE']
else:
    print "Don't panic! "+user+" not set up for pathname settings"
        


karen


In [4]:
def appendNC2D(ncname,xlen,deltamod,deltaobs,deltant_mod,deltant_obs,time3d,delta3d_mod,delta3d_obs):

    """ 
    Appends a netcdf file with new variable
    """
    f = netcdf.netcdf_file(ncname, 'a')
    
    f.createDimension('time3day',len(time3d))

    delta_mod = f.createVariable('delta_mod', 'f', ('time',))
    delta_mod[:] = deltamod
    delta_obs = f.createVariable('delta_obs', 'f', ('time',))
    delta_obs[:] = deltaobs
    delta_nt_mod = f.createVariable('delta_nt_mod', 'f', ('time',))
    delta_nt_mod[:] = deltant_mod
    delta_nt_obs = f.createVariable('delta_nt_obs', 'f', ('time',))
    delta_nt_obs[:] = deltant_obs
    delta_3d_mod = f.createVariable('delta_3d_mod', 'f', ('time3day',))
    delta_3d_mod[:] = delta3d_mod
    delta_3d_obs = f.createVariable('delta_3d_obs', 'f', ('time3day',))
    delta_3d_obs[:] = delta3d_obs
                                      
    time3day = f.createVariable('time3day', 'f', ('time3day',))
    time3day[:] = time3d
    time3day.units = 'days since 1970-01-01'


for index in range(len(MOORING_NAMES)):
    MOORING_NAME = MOORING_NAMES[index]
    print(MOORING_NAME)
    
    # AMM60
    print('AMM60')
    MOORING_filename = '/media/data1/AMM60/FASTNEt/AMM60/Orig/mooring_' + MOORING_NAME + '.nc'
    temp_obs = amm60_data_tools.readMODELnc(MOORING_filename,'temp_obs')
    temp_mod = amm60_data_tools.readMODELnc(MOORING_filename,'temp_mod')
    depth = amm60_data_tools.readMODELnc(MOORING_filename,'depth')
    time_counter = amm60_data_tools.readMODELnc(MOORING_filename,'time')*86400
    [delta_mod, delta_nt_mod, delta_var, time_datetime,  delta_runwin_mod, delta_var_runwin, time_datetime_runwin] = amm60_data_tools.delta_diagnose(temp_mod, time_counter, depth, 100 )
    [delta_obs, delta_nt_obs, delta_var, time_datetime,  delta_runwin_obs, delta_var_runwin, time_datetime_runwin] = amm60_data_tools.delta_diagnose(temp_obs, time_counter, depth, 100 )
    time_sec = np.zeros(len(time_datetime_runwin))
    for idx, val in enumerate(time_datetime_runwin):
        time_str = datetime.strftime(time_datetime_runwin[idx], '%Y-%m-%d %H:    %M:%S')
        d = datetime.strptime(time_str, '%Y-%m-%d %H:    %M:%S')
        time_sec[idx] = time.mktime(d.timetuple())
    
    appendNC2D('/media/data1/AMM60/FASTNEt/AMM60/mooring_' + MOORING_NAME + '.nc',len(time_counter),delta_mod,delta_obs,delta_nt_mod,delta_nt_obs,time_sec/86400,delta_runwin_mod,delta_runwin_obs)
    
    # AMM7
    print('AMM7')
    MOORING_filename = '/media/data1/AMM60/FASTNEt/AMM7/Orig/mooring_' + MOORING_NAME + '.nc'
    temp_obs = amm60_data_tools.readMODELnc(MOORING_filename,'temp_obs')
    temp_mod = amm60_data_tools.readMODELnc(MOORING_filename,'temp_mod')
    depth = amm60_data_tools.readMODELnc(MOORING_filename,'depth')
    time_counter = amm60_data_tools.readMODELnc(MOORING_filename,'time')*86400
    [delta_mod, delta_nt_mod, delta_var, time_datetime,  delta_runwin_mod, delta_var_runwin, time_datetime_runwin] = amm60_data_tools.delta_diagnose(temp_mod, time_counter, depth, 100 )
    [delta_obs, delta_nt_obs, delta_var, time_datetime,  delta_runwin_obs, delta_var_runwin, time_datetime_runwin] = amm60_data_tools.delta_diagnose(temp_obs, time_counter, depth, 100 )
    time_sec = np.zeros(len(time_datetime_runwin))
    for idx, val in enumerate(time_datetime_runwin):
        time_str = datetime.strftime(time_datetime_runwin[idx], '%Y-%m-%d %H:    %M:%S')
        d = datetime.strptime(time_str, '%Y-%m-%d %H:    %M:%S')
        time_sec[idx] = time.mktime(d.timetuple())
    
    appendNC2D('/media/data1/AMM60/FASTNEt/AMM7/mooring_' + MOORING_NAME + '.nc',len(time_counter),delta_mod,delta_obs,delta_nt_mod,delta_nt_obs,time_sec/86400,delta_runwin_mod,delta_runwin_obs)

        
    # NNA
    print('NNA')
    MOORING_filename = '/media/data1/AMM60/FASTNEt/NNA/Orig/mooring_' + MOORING_NAME + '.nc'
    temp_obs = amm60_data_tools.readMODELnc(MOORING_filename,'temp_obs')
    temp_mod = amm60_data_tools.readMODELnc(MOORING_filename,'temp_mod')
    depth = amm60_data_tools.readMODELnc(MOORING_filename,'depth')
    time_counter = amm60_data_tools.readMODELnc(MOORING_filename,'time')*86400
    [delta_mod, delta_nt_mod, delta_var, time_datetime,  delta_runwin_mod, delta_var_runwin, time_datetime_runwin] = amm60_data_tools.delta_diagnose(temp_mod, time_counter, depth, 100 )
    [delta_obs, delta_nt_obs, delta_var, time_datetime,  delta_runwin_obs, delta_var_runwin, time_datetime_runwin] = amm60_data_tools.delta_diagnose(temp_obs, time_counter, depth, 100 )
    time_sec = np.zeros(len(time_datetime_runwin))
    for idx, val in enumerate(time_datetime_runwin):
        time_str = datetime.strftime(time_datetime_runwin[idx], '%Y-%m-%d %H:    %M:%S')
        d = datetime.strptime(time_str, '%Y-%m-%d %H:    %M:%S')
        time_sec[idx] = time.mktime(d.timetuple())
    
    appendNC2D('/media/data1/AMM60/FASTNEt/NNA/mooring_' + MOORING_NAME + '.nc',len(time_counter),delta_mod,delta_obs,delta_nt_mod,delta_nt_obs,time_sec/86400,delta_runwin_mod,delta_runwin_obs)
    

ST1
AMM60
Chunking done.
Chunking done.
AMM7
Chunking done.
Chunking done.
NNA
Chunking done.
Chunking done.
ST2
AMM60
Chunking done.
Chunking done.
AMM7
Chunking done.
Chunking done.
NNA
Chunking done.
Chunking done.
ST4
AMM60
Chunking done.
Chunking done.
AMM7
Chunking done.
Chunking done.
NNA
Chunking done.
Chunking done.
ST5
AMM60
Chunking done.
Chunking done.
AMM7
Chunking done.
Chunking done.
NNA
Chunking done.
Chunking done.
SE
AMM60
Chunking done.
Chunking done.
AMM7
Chunking done.
Chunking done.
NNA
Chunking done.
Chunking done.


[  1.33863306e+09   1.33889226e+09   1.33915146e+09   1.33941066e+09
   1.33966986e+09   1.33992906e+09   1.34018826e+09   1.34044746e+09
   1.34070666e+09   1.34096586e+09]
